In [57]:
from datetime import datetime
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm
from time import sleep
import os



Geopy no lo suo al final

In [ ]:
# geolocator = Nominatim(user_agent="my_application") #Primero hice un ejemplo con uno
# location = geolocator.geocode("santa cruz de tenerife")
# print(location.address)
# print((location.latitude, location.longitude))
# print(location.raw)

# lat= location.latitude
# lng= location.longitude
# print(lat)
# print(lng)

**API de Hoteles**   habrá que hacer que en lugar de palma de mayorca se ponga CANARIAS

In [44]:


url_hotel = "https://tripadvisor-scraper.p.rapidapi.com/hotels/list"

querystring = {f"query":"palma de mallorca","page":"2"}

headers = {
	"x-rapidapi-key": "978309fe9dmshe0626a77c211be6p1786a7jsne7e8845b8cfa",
	"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
}

response = requests.get(url_hotel, headers=headers, params=querystring)
responsa=response.json()
df= pd.DataFrame(responsa["results"])
df

In [41]:
df= pd.DataFrame(responsa["results"])
df.head(2)

,id,name,link,reviews,rating,price_range_usd,phone,address,detailed_address,ranking,featured_image,latitude,longitude,amenities,providers
0,1797289,Hotel Feliz,https://www.tripadvisor.com/Hotel_Review-g1874...,854,4.5,"{'min': 108, 'max': 260}",+34 971 28 88 47,"Avinguda de Joan Miro, 74, 07015 Palma de Mall...","{'street': 'Avinguda de Joan Miro, 74', 'city'...","{'current_rank': 1, 'total': 21}",None,39.560840,2.624639,"[Free Wifi, Outdoor pool, Restaurant, Pets All...","[Booking.com, Expedia.com, Hotels.com, ZenHote..."
1,620006,MHOUSE Hotel,https://www.tripadvisor.com/Hotel_Review-g1874...,225,4.5,"{'min': 166, 'max': 298}",+34 971 21 48 48,"Calle de Can Maçanet 1A, 07003 Palma de Mallor...","{'street': 'Calle de Can Maçanet 1A', 'city': ...","{'current_rank': 62, 'total': 95}",None,39.574432,2.651357,"[Free Wifi, Outdoor pool, Restaurant, Bar/Lounge]","[Algotels, Trip.com, StayForLong, Booking.com]"


In [47]:
#Hacer archivo json para abrir dicc de hoteles

**API de los vuelos(TENE (TNF Y TNS))**  Meter los códigos de aeropuerto que necesitemos

In [54]:
url = "https://sky-scrapper.p.rapidapi.com/api/v2/flights/searchFlights"

querystring = {"originSkyId":"MAD","destinationSkyId":"TENE","originEntityId":"95565077","destinationEntityId":"27547165","date":"2024-11-03","returnDate":"2024-11-06","cabinClass":"economy","adults":"1","sortBy":"best","currency":"EUR"}

headers = {
	"x-rapidapi-key": "978309fe9dmshe0626a77c211be6p1786a7jsne7e8845b8cfa",
	"x-rapidapi-host": "sky-scrapper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

responsa=response.json()



,day,group,price
0,2024-10-18,high,98.78
1,2024-10-19,high,100.29
2,2024-10-20,high,96.57
3,2024-10-21,high,71.65
4,2024-10-22,high,33.20
...,...,...,...
341,2025-10-10,high,28.63
342,2025-10-13,high,42.33
343,2025-10-14,high,32.06
344,2025-10-15,high,28.63


In [50]:
lista_codigos_aeropuertos= ["MAD", "PMI", "TFN","GMZ", "MAH"]

web para mirar los códigos:   


https://rapidapi.com/apiheya/api/sky-scrapper/playground/apiendpoint_efcce954-2563-41cb-97fc-b2abe7851b3d

**API de los vuelos(vuelta)**  Meter los códigos de aeropuerto que necesitemos

In [3]:
url = "https://sky-scrapper.p.rapidapi.com/api/v2/flights/searchFlights"

querystring = {"originSkyId":"MAD","destinationSkyId":"PMI","originEntityId":"95565077","destinationEntityId":"95565111","date":"2024-11-03","returnDate":"2024-11-06","cabinClass":"economy","adults":"1","sortBy":"best","limit":"50","currency":"EUR"}

headers = {
	"x-rapidapi-key": "978309fe9dmshe0626a77c211be6p1786a7jsne7e8845b8cfa",
	"x-rapidapi-host": "sky-scrapper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

responsa=response.json()


In [33]:


with open('archivo.json', 'w') as archivo:
    json.dump(responsa, archivo, indent=4)

In [98]:

res2 = res["itineraries"][0]["legs"][0]["origin"]
# res3 = res["itineraries"][0]["legs"][0]["destination"]



In [8]:
res= responsa["data"]
precio= res["itineraries"][0]["price"]["raw"]

id_origen= res["itineraries"][0]["legs"][0]["origin"]["id"]
origen= res["itineraries"][0]["legs"][0]["origin"]["city"]

id_destino= res["itineraries"][0]["legs"][0]["destination"]["id"]
destino= res["itineraries"][0]["legs"][0]["destination"]["city"]

duracion_minuts= res["itineraries"][0]["legs"][0]["durationInMinutes"]
escalas= res["itineraries"][0]["legs"][0]["stopCount"]

salida= res["itineraries"][0]["legs"][0]["departure"]
llegada= res["itineraries"][0]["legs"][0]["arrival"]

aerolinea= res["itineraries"][0]["legs"][0]["carriers"]["marketing"][0]["name"]

In [9]:


dicc= {"id_origen":id_origen,
       "ciudad_origen":origen,
       "id_destino": id_destino,
       "ciudad_destino":destino,
       "salida":salida,
       "llegada":llegada,
       "duracion(min)":duracion_minuts,
       "escalas":escalas,
       "aerolinea":aerolinea,
       "precio":precio
        }
dicc

{'id_origen': 'MAD',
 'ciudad_origen': 'Madrid',
 'id_destino': 'PMI',
 'ciudad_destino': 'Palma',
 'salida': '2024-11-03T08:35:00',
 'llegada': '2024-11-03T10:05:00',
 'duracion(min)': 90,
 'escalas': 0,
 'aerolinea': 'Ryanair',
 'precio': 48.66}

Sacar vuelos ida y vuelta a PMI

In [47]:
lista_dicc=[]

for vuelo in range(0,len(res["itineraries"])):
        res= responsa["data"]
        precio= res["itineraries"][vuelo]["price"]["raw"]
        for i in range(0,2):
                id_origen= res["itineraries"][vuelo]["legs"][i]["origin"]["id"]
                origen= res["itineraries"][vuelo]["legs"][i]["origin"]["city"]

                id_destino= res["itineraries"][vuelo]["legs"][i]["destination"]["id"]
                destino= res["itineraries"][vuelo]["legs"][i]["destination"]["city"]

                duracion_minuts= res["itineraries"][vuelo]["legs"][i]["durationInMinutes"]
                escalas= res["itineraries"][vuelo]["legs"][i]["stopCount"]

                salida= res["itineraries"][vuelo]["legs"][i]["departure"]
                llegada= res["itineraries"][vuelo]["legs"][i]["arrival"]

                aerolinea= res["itineraries"][vuelo]["legs"][i]["carriers"]["marketing"][0]["name"]

                dicc= {"id_origen":id_origen,
                "ciudad_origen":origen,
                "id_destino": id_destino,
                "ciudad_destino":destino,
                "salida":salida,
                "llegada":llegada,
                "duracion(min)":duracion_minuts,
                "escalas":escalas,
                "aerolinea":aerolinea,
                "precio":precio
                }
                lista_dicc.append(dicc)


In [49]:
df_vuelos_ida_vuelta_PMI= pd.DataFrame(lista_dicc)
df_vuelos_ida_vuelta_PMI.head()

,id_origen,ciudad_origen,id_destino,ciudad_destino,salida,llegada,duracion(min),escalas,aerolinea,precio
0,MAD,Madrid,PMI,Palma,2024-11-03T08:35:00,2024-11-03T10:05:00,90,0,Ryanair,48.66
1,PMI,Palma,MAD,Madrid,2024-11-06T22:20:00,2024-11-06T23:50:00,90,0,Ryanair,48.66
2,MAD,Madrid,PMI,Palma,2024-11-03T08:35:00,2024-11-03T10:05:00,90,0,Ryanair,69.69
3,PMI,Palma,MAD,Madrid,2024-11-06T10:55:00,2024-11-06T12:25:00,90,0,Ryanair,69.69
4,MAD,Madrid,PMI,Palma,2024-11-03T07:05:00,2024-11-03T08:30:00,85,0,Iberia Express,46.60


In [50]:
df_vuelos_ida_vuelta_PMI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_origen       20 non-null     object 
 1   ciudad_origen   20 non-null     object 
 2   id_destino      20 non-null     object 
 3   ciudad_destino  20 non-null     object 
 4   salida          20 non-null     object 
 5   llegada         20 non-null     object 
 6   duracion(min)   20 non-null     int64  
 7   escalas         20 non-null     int64  
 8   aerolinea       20 non-null     object 
 9   precio          20 non-null     float64
dtypes: float64(1), int64(2), object(7)
memory usage: 1.7+ KB


In [ ]:
df_vuelos_ida_vuelta_PMI[salida].to_da

In [37]:
len(res["itineraries"])


10